# Train ADAGE models on RNAseq data

### Georgia Doing 2021

This notebook walks through the training of an RNAseq-based ADAGE model as a continuation of analyses from last year (2021_06_eADAGE_adapt/seqADAGE).

Since then we have downloaded a new compendium of RNAseq data and aligned it to the PAO1 and PA14 reference genomes using prokarytoic-optimized parameters.
(https://github.com/hoganlab-dartmouth/pa-seq-compendia)

Section outline:

### 1. Setup and Load data
    
    1.1 libraries and reload for dev
    1.2 load ADAGE trained on array data with Jie's original code
### 2. Train models with different param options

    2.1 USing seed 560, train on different params
    2.2 Load models trained with seed 560, on different params, from files
### 3. Train replicate models for ensemble
    3.1 Use seed 661 with different params
    3.2 Use seeds 662-669 to see how models with same params look
    3.3 Train models with original ADAGE params
    3.4 Train with chosen params, seeds up to 759 for ensemble
### 4. Try for deep count autoencoder

## 1. Setup and load data

In [ ]:
import run_count_autoencoder
import run_model
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from imp import reload
import Adage
from scipy.stats import hypergeom
import csv
import TiedWeightsEncoder

In [ ]:
# for dev purposes, while updating run_model.py need to reload
reload(run_count_autoencoder)
reload(run_model)
reload(Adage)
reload(TiedWeightsEncoder)

### 1.1 Load data

In [ ]:
array_comp = pd.read_csv(open('../data_files/train_set_normalized.csv', "rb"),index_col=0)
seq_comp_floor = pd.read_csv(open('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log.csv', "rb"),index_col=0)
seq_comp = pd.read_csv(open('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv', "rb"),index_col=0)
print(seq_comp.shape)
print(seq_comp_floor.shape)
print(array_comp.shape)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(16 ,4))
sns.histplot(array_comp.to_numpy().flatten(),binwidth=0.01, stat='density',ax=ax1)
sns.histplot(seq_comp.to_numpy().flatten(),binwidth=0.01,stat='density',ax=ax2)
sns.histplot(seq_comp_floor.to_numpy().flatten(), bins=200,stat='density',ax=ax3)

### 1.2 Load ADAGE model trained using Jie's original code

In [ ]:
m = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log.csv',lr=.0001,seed=460,kl1=1e-10,kl2=1e-1, act = 'sigmoid', tied = True, epochs=5, init='glorot_uniform', batch_size=10, v=0)
tf_adage = Adage.Adage(m.autoencoder, m.history, m.compendium)
tf_weights = np.array(pd.read_csv('../outputs/ADAGE_OG_weights.csv', header = None))
tf_cost = pd.read_csv('../outputs/ADAGE_OG_cost_log.csv')
tf_adage.weights = tf_weights
tf_adage.loss = tf_cost['cost']
tf_adage.set_hwg_cutoff(2.5).shape

In [ ]:
model_dict = {
    "tf_adage": tf_adage,
    "k_adage": m
}
fig, ax = plt.subplots(1, 2,figsize=(16 ,4))
fig.tight_layout(pad=3.0)

name = 'tf_adage'
model_temp = model_dict[name]
ax[0].plot(list(range(0,5)), model_temp.loss[0:5], linewidth=1, markersize=2, color = 'orange')
ax[0].set(title = name, xlabel = 'Epochs', ylabel = 'Loss')
ax[0].set(title = name)

name = 'k_adage'
model_temp = model_dict[name]
ax[1].plot(list(range(0,5)), model_temp.loss, linewidth=1, markersize=2, color = 'orange')
ax[1].plot(list(range(0,5)), model_temp.val_loss, linewidth=1, markersize=2, color = 'red')
ax[1].set(title = name, xlabel = 'Epochs', ylabel = 'Loss')
ax[1].set(title = name)

## 2. Train models to compare paramters (seeds starting at 550)

### 2.1 Train models with seed 550 (or skip to 2.1 to load from files)

In [ ]:
# skip this chunk to load models from files, go to chunk below
inits = ['glorot_normal']
L1_norm = [1e-10] # 
L2_norm = [0, 1e-5, 1e-10]
act_fun = ['sigmoid', 'tanh','relu']
tied = [True, False]
lr = [1.5,1.0,0.5,0.1,0.01,0.001,0.0001] 

model_dict_da = {
    "tf_adage": tf_adage
}
model_dict_dca = {
    "tf_adage": tf_adage
}
model_dict_dca01 = {
    "tf_adage": tf_adage
}
model_dict_arr = {
    "tf_adage": tf_adage
}
model_dict_floor = {
    "tf_adage": tf_adage
}

for seed in range(1):
    for w in tied:
        for i in inits:
            #print(i)
            for l in L1_norm:
                #print(l)
                for n in L2_norm:
                    for a in act_fun:
                       # print(a)
                        for t in lr:
                            #print(t)
                            name = 'ad_' + i + '_' + str(l) + '_' + str(n) + '_' + a + '_tied' + '_' + str(w) + '_' + '_lr' + str(t) 
                            print(name)
                            m_arr = run_model.run_model('../data_files/train_set_normalized.csv',seed=seed+560,lr=t,kl1=l, kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            m_dca01 = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            m_dca = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            m_dca_floor = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_floor.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)


                            model_dict_arr[name] = m_arr
                            model_dict_da[name] = m_da
                            model_dict_dca01[name] = m_dca01
                            model_dict_dca[name] = m_dca
                            model_dict_floor[name] = m_dca_floor

### 2.2 Or try to load models from files first, to save time re-training param combos already tested (seed still 550)

In [ ]:
inits = ['glorot_uniform','glorot_normal']
L1_norm = [0, 1e-5, 1e-10] # 
L2_norm = [0, 1e-5, 1e-10]
act_fun = ['sigmoid', 'tanh','relu']
tied = [True, False]
lr = [1.5,1.0,0.5,0.1,0.01,0.001,0.0001] 

model_dict_da = {
    "tf_adage": tf_adage
}
model_dict_dca = {
    "tf_adage": tf_adage
}
model_dict_dca01 = {
    "tf_adage": tf_adage
}
model_dict_arr = {
    "tf_adage": tf_adage
}
model_dict_floor = {
    "tf_adage": tf_adage
}


for w in tied:
    for i in inits:
        for l in L1_norm:
            for n in L2_norm:
                for a in act_fun:
                    for t in lr:
                        seed = str(560)
                        name = 'ad_' + i + '_' + str(l) + '_' + str(n) + '_' + a + '_tied' + '_' + str(w) + '_' + '_lr' + str(t) 
                        #print(name)
                    #print(t)
                    #m = run_model.run_model('data_files/train_set_normalized.csv',seed=460,kl1=l, act = a, tied = t, epochs=500, init=i)
                        prefix = '/data_files/train_set_normalized_seed:' + seed + '_kl1:' + str(l) + '_kl2:0_act:' + a + '_init:' + i + '_ep:50_tied:' + str(w) + '_batch:10_lr:' + str(t)
                    #print(prefix)
                        try:
                            weights_temp = np.array(pd.read_csv('../outputs/weights/' + prefix + '_en_weights_da.csv', header = None))
                            loss_temp = np.array(pd.read_csv('../outputs/loss/' + prefix + '_loss_da.csv', header = None))
                            val_loss_temp = np.array(pd.read_csv('../outputs/val_loss/' + prefix + '_val_loss_da.csv', header = None))
                            adage_temp = Adage.Adage(tf_adage.autoencoder, tf_adage.history, tf_adage.compendium)
                            adage_temp.weights = weights_temp
                            adage_temp.loss = loss_temp[0]
                            adage_temp.val_loss = val_loss_temp[0]
                            adage_temp.set_hwg_cutoff(2.5)
                        #name = 'ad_' + i + '_' + str(l) + '_' + a + '_tied' + str(t) + seed
                    #print(name)
                            model_dict_da[name] = adage_temp
                        except:
                            name = 'ad_' + i + '_' + str(l) + '_' + str(n) + '_' + a + '_tied' + '_' + str(w) + '_' + '_lr' + str(t) 
                            print(name)
                            seed=0
                            m_arr = run_model.run_model('../data_files/train_set_normalized.csv',seed=seed+560,lr=t,kl1=l, kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            m_dca01 = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            m_dca = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            m_dca_floor = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_floor.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)


                            model_dict_arr[name] = m_arr
                            model_dict_da[name] = m_da
                            model_dict_dca01[name] = m_dca01
                            model_dict_dca[name] = m_dca
                            model_dict_floor[name] = m_dca_floor

## 3. Setting the seed to 660 to train a new set of models

### 3.1 First, try different params again with seed 660 to make sure all looks same as seed 560

In [ ]:
inits = ['glorot_uniform']
L1_norm = [0, 1e-5, 1e-10,1e-15]
L2_norm = [0]
act_fun = ['sigmoid', 'tanh','relu']
tied = [True]
lr = [1.0,0.5,0.1,0.01] #1.5,

for seed in range(1):
    for w in tied:
        for i in inits:
            #print(i)
            for l in L1_norm:
                #print(l)
                for n in L2_norm:
                    for a in act_fun:
                       # print(a)
                        for t in lr:
                            #print(t)
                            name = 'ad_' + i + '_' + str(l) + '_' + str(n) + '_' + a + '_tied' + '_' + str(w) + '_' + '_lr' + str(t) 
                            print(name)
                            m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv',seed=seed+660,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=250, init=i, batch_size = 10,v=0)
                                

### 3.2 Now, train 8 new models with seeds 662 - 670
(starting with 8 to look at enrichments quick before scaling up even more)

In [ ]:
inits = ['glorot_uniform']
L1_norm = [0, 1e-5, 1e-10,1e-15]
L2_norm = [0]
act_fun = ['sigmoid', 'tanh','relu']
tied = [True]
lr = [1.0,0.5,0.1,0.01] #1.5,

for seed in range(2,10):
    for w in tied:
        for i in inits:
            #print(i)
            for l in L1_norm:
                #print(l)
                for n in L2_norm:
                    for a in act_fun:
                       # print(a)
                        for t in lr:
                            #print(t)
                            name = 'ad_' + i + '_' + str(l) + '_' + str(n) + '_' + a + '_tied' + '_' + str(w) + '_' + '_lr' + str(t) 
                            print(name)
                            #m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01_bygene.csv',seed=seed+660,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=250, init=i, batch_size = 10,v=0)
                            m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01_byall.csv',seed=seed+660,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=250, init=i, batch_size = 10,v=0)
                            #m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv',seed=seed+660,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=250, init=i, batch_size = 10,v=0)


### 3.3 Train some models with original params and seed 660 
Note this data is normalized by gene, activation is sigmoid - matched original parameters of ADAGE

* hidden features: 300
* epochs: 250
* curruption: 0.01
* batchs zie: 10
* validatoin split: 0.1
* activation: sigmoid
* objective: binary cross-entropy
* learning rate: 0.1
* L1: 1e-10
* tied: True
* init: uniform

In [ ]:
for seed in range(1):
    #m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01_bygene.csv',seed=seed+660,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=250, init=i, batch_size = 10,v=0)
    run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01_bygene.csv',seed=seed+660,lr=1.0,kl1=1e-10,kl2=0, act = 'sigmoid', tied = True, epochs=250, init='glorot_uniform', batch_size = 10,v=0)
    #m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv',seed=seed+660,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=250, init=i, batch_size = 10,v=0)


### 3.4 Now, finally,  train the models to get a collection of 100 models with seeds 660 - 759
(this loop previously ran range(0,75)

* hidden features: 300
* epochs: 250
* curruption: 0.01
* batchs zie: 10
* validatoin split: 0.1
* activation: relu
* objective: binary cross-entropy
* learning rate: 0.1
* L1: 1e-5
* tied: True
* init: uniform

In [ ]:
for seed in range(76,100):
    #m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01_bygene.csv',seed=seed+660,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=250, init=i, batch_size = 10,v=0)
    run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01_byall.csv',seed=seed+660,lr=0.1,kl1=1e-5,kl2=0, act = 'relu', tied = True, epochs=250, init='glorot_uniform', batch_size = 10,v=0)
    #m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv',seed=seed+660,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=250, init=i, batch_size = 10,v=0)


## 4. Try DCA (count autoencoder)
seed back to 560

In [ ]:
inits = ['glorot_uniform']
L1_norm = [0, 1e-10,1e-15] #1e-5,
L2_norm = [1e-5,1e-10]
act_fun = [ 'sigmoid','tanh','relu']#
tied = [True]
lr = [1.5,1.0,0.5,0.1,0.01,0.001] #

for seed in range(1):
    for w in tied:
        for i in inits:
            #print(i)
            for l in L1_norm:
                #print(l)
                for n in L2_norm:
                    for a in act_fun:
                       # print(a)
                        for t in lr:
                            #print(t)
                            name = 'ad_' + i + '_' + str(l) + '_' + str(n) + '_' + a + '_tied' + '_' + str(w) + '_' + '_lr' + str(t) 
                            print(name)
                            m_da = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01_byall.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            m_da_bg = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01_bygene.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            #m_da_bs = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_TPM_log_01_bysamp.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)


In [ ]:
inits = ['glorot_normal','glorot_uniform']
L1_norm = [0, 1e-5, 1e-10] # 
L2_norm = [0, 1e-5, 1e-10]
act_fun = ['sigmoid', 'tanh','relu']
tied = [True, False]
lr = [1.5,1.0,0.5,0.1,0.01,0.001,0.0001] 

model_dict_da = {
    "tf_adage": tf_adage
}
model_dict_dca = {
    "tf_adage": tf_adage
}
model_dict_dca01 = {
    "tf_adage": tf_adage
}
model_dict_arr = {
    "tf_adage": tf_adage
}
model_dict_floor = {
    "tf_adage": tf_adage
}

for seed in range(1):
    for w in tied:
        for i in inits:
            #print(i)
            for l in L1_norm:
                #print(l)
                for n in L2_norm:
                    for a in act_fun:
                       # print(a)
                        for t in lr:
                            #print(t)
                            name = 'ad_' + i + '_' + str(l) + '_' + str(n) + '_' + a + '_tied' + '_' + str(w) + '_' + '_lr' + str(t) 
                            print(name)
                            m_arr = run_model.run_model('../data_files/train_set_normalized.csv',seed=seed+560,lr=t,kl1=l, kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            m_da = run_model.run_model('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            #m_dca01 = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log_01.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            #m_dca = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_log.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)
                            #m_dca_floor = run_count_autoencoder.run_count_autoencoder('../data_files/pao1_aligned_rnaseq_compendium_zp2_MRnorm_floor.csv',seed=seed+560,lr=t,kl1=l,kl2=n, act = a, tied = w, epochs=50, init=i, batch_size = 10,v=0)


                            model_dict_arr[name] = m_arr
                            model_dict_da[name] = m_da
                            #model_dict_dca01[name] = m_dca01
                            #model_dict_dca[name] = m_dca
                            #model_dict_floor[name] = m_dca_floor

In [ ]:
print(model_dict_da.keys())
print(model_dict_dca.keys())

In [ ]:
yd = len(inits) * len(L1_norm)
xd = len(act_fun) * len(lr)
fig, ax = plt.subplots(xd, yd,figsize=(xd*4 ,yd *4))
fig.tight_layout(pad=3.0)

xi = 0
yi = 0
for l in L1_norm:
    for i in inits:
        xi = 0
        for a in act_fun:
            for t in lr:
                name = 'ad_' + i + '_' + str(l) + '_' + a + '_tied' + str(t) 
                #print(name)
                model_temp = model_dict_da[name]
                ax[xi,yi].plot(list(range(0,50)), model_temp.loss, linewidth=1, markersize=2, color = 'orange')
                ax[xi,yi].set(title = name, xlabel = 'Epochs', ylabel = 'Loss')
                xi = xi+1
        yi=yi+1


    

In [ ]:
yd = len(inits) * len(L1_norm)
xd = len(act_fun) * len(lr)
fig, ax = plt.subplots(xd, yd,figsize=(xd*4 ,yd *4))
fig.tight_layout(pad=3.0)

xi = 0
yi = 0
for l in L1_norm:
    for i in inits:
        xi = 0
        for a in act_fun:
            for t in lr:
                name = 'ad_' + i + '_' + str(l) + '_' + a + '_tied' + str(t) 
                #print(name)
                model_temp = model_dict_dca[name]
                ax[xi,yi].plot(list(range(0,50)), model_temp.loss, linewidth=1, markersize=2, color = 'orange')
                ax[xi,yi].set(title = name, xlabel = 'Epochs', ylabel = 'Loss')
                xi = xi+1
        yi=yi+1


    